In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("kc_house_data.csv")

In [4]:
pd.options.display.max_columns=100

In [5]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

### Data Preparation

In [8]:
df = df.drop(columns=["id", "date"])

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train_df, test_df = train_test_split(df, train_size=0.8)

In [11]:
train_df.shape, test_df.shape

((17290, 19), (4323, 19))

In [12]:
x_train = train_df.drop(columns="price")
y_train = train_df.price

x_test = test_df.drop(columns="price")
y_test = test_df.price

The scale of the target variable is huge so let's normalize it for convinience.

In [13]:
from sklearn.preprocessing import MinMaxScaler

In [19]:
scaler = MinMaxScaler()

y_train = y_train.values.reshape((-1, 1))

y_train = scaler.fit_transform(y_train)

y_test = y_test.values.reshape((-1, 1))

y_test = scaler.transform(y_test)

### Modelling

In [22]:
import xgboost as xgb

In [23]:
model = xgb.XGBRegressor()

model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [24]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [25]:
r2_score(y_train, model.predict(x_train))

0.9752921312158802

In [26]:
r2_score(y_test, model.predict(x_test))

0.8897805007495001

#### The model shows some amount of overfitting

### Hyperparameter Tuning

In [27]:
from sklearn.model_selection import GridSearchCV

In [30]:
# we are training for very few hyperparameter values for demonstration

params = {
    "learning_rate" : [0.1, 0.2],
    "reg_alpha" : [0.1, 0.2],
    "reg_lambda" : [0.1, 0.2]
}

model_basic = xgb.XGBRegressor()

gs = GridSearchCV(estimator=model_basic,
                 param_grid=params,
                 n_jobs=-1,
                 cv=4)

In [31]:
gs.fit(x_train, y_train)

GridSearchCV(cv=4,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,


In [32]:
model_best = gs.best_estimator_

r2_score(y_train, model_best.predict(x_train))

0.9571328340551115

In [33]:
r2_score(y_test, model_best.predict(x_test))

0.910490799587253

#### We are able to achieve better generalization using GridSearchCV